In [ ]:
# imports
from snowexsql.db import get_db
from snowexsql.data import SiteData, ImageData, LayerData, PointData
from snowexsql.conversions import query_to_geopandas
import datetime

In [ ]:
# load the database
db_name = 'snow:hackweek@52.32.183.144/snowex'
engine, session = get_db(db_name)
print('snowexsql database successfully loaded')

In [ ]:
# Great for debugging especially when trying different queries
# session.rollback()

## Snow Pits --> LayerData & SiteData in the snowexsql database
### Example 1: 
Let's find all the pits that overlap with UAVSAR data

1a). Unsure of the fly over date, but know which instrument you'd like to overlap with, here's how:

In [ ]:
# Form on the Images table that returns Raster collection dates
qry = session.query(ImageData.date)

# Filter for UAVSAR data
qry = qry.filter(ImageData.surveyors == 'UAVSAR team, JPL')

# Grab the unique dates
qry = qry.distinct()
qry
# Execute the query 
dates = qry.all() 

# Clean up the dates to remove the tuple
dates = [d[0] for d in dates] 

# Find all the snow pits done on these days and return a geopandas df
qry = session.query(SiteData.geom, SiteData.site_id, SiteData.date)
qry = qry.filter(SiteData.date.in_(dates))
df = query_to_geopandas(qry, engine)
df.head()

# Close your session to avoid hanging transactions
session.close()

1b). Want to select from a range of dates near the fly over date? Here's how:

In [ ]:
# Pick a day from the list of dates
dt = dates[0] 

# Form a date range to query on either side of our chosen day 
date_range = [dt + i * datetime.timedelta(days=1) for i in [-1, 0, 1]]

# Find all the snow pits done on these days and return a geopandas df
qry = session.query(SiteData.geom, SiteData.site_id, SiteData.date)
qry = qry.filter(SiteData.date.in_(date_range))
df = query_to_geopandas(qry, engine)
df

1c). Have a known date in mind that you wish to select, here's how:

In [ ]:
# Find all the data that was collected on 2-12-2020
dt = datetime.date(2020, 2, 12)

# Grab all Point data instruments from our date
point_instruments = session.query(PointData.instrument).filter(PointData.date == dt).distinct().all()
# print('Point Data are: ', point_instruments)

# Grab all Layer data instruments from our date
layer_instruments = session.query(LayerData.instrument, LayerData.type).filter(LayerData.date == dt).distinct().all()
# print(layer_instruments)

# Grab all Image data instruments from our date
image_instruments = session.query(ImageData.instrument).filter(ImageData.date == dt).distinct().all()
# print(image_instruments)

qry = session.query(PointData.geom, PointData.instrument, PointData.site_id, PointData.date)
qry = qry.filter(PointData.date == dt) 
df = query_to_geopandas(qry, engine)
df

In [ ]:
dt = datetime.date(2020, 2, 12)
dt

### Your turn
#### try this: query for a different airborne instrument (hint: airborne lidar--> 'ASO Inc.')

In [ ]:
#

In [ ]:
#RANDOM QUESTIONS/ CODE EXAMPLE BOX
qry = session.query(ImageData.surveyors) # can I print a list of surveyor options??
surveyors_lst = session.query(ImageData.surveyors).distinct().all()
surveyors_lst
# qry = session.query(SiteData.site_id, SiteData.date)
# qry = qry.filter(SiteData.date.in_(dates))
# results = qry.all()

### Example 2: 
grab all pits based on flight line pattern

1). only want to analyize pits from the 'N' flight line?

2). what about the 'N' flight line for a 1 week period?

In [ ]:
# Grab density
import matplotlib.pyplot as plt 

session.rollback()

q = session.query(LayerData).filter(LayerData.type == 'density').filter(LayerData.site_id.contains('N')).limit(100)
df = query_to_geopandas(q, engine)
# Convert density to float
df['value'] = df['value'].astype(float)

df 

df.plot()
plt.show()


# Calculate SWE
# swe_lambda = lambda row: row['value'] * (row['depth'] - row['bottom_depth']) / 100
# df['swe'] = df.apply(swe_lambda, axis=1)

In [ ]:
df

### Example 3: 
grab all pits based on the depth and vegetation matrix

1). interested in isolating shallow pits? here's how: 

2). what about pits in dense canopy cover? 

## Depths
### Example 1:
There are a lot of different instruments that collected snow depth data. Say you are interested in working with a certain instrument, here's how: